In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
#from src import detect_faces, show_bboxes
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from PIL import Image

import torch
from torchvision import transforms, datasets
import numpy as np
import os

In [4]:
!mkdir Dataset

In [5]:
!mkdir Dataset/emotiw

In [6]:
!mkdir Dataset/emotiw/test

In [7]:
!unzip /content/drive/MyDrive/GAF_3.0.zip

Archive:  /content/drive/MyDrive/GAF_3.0.zip
 extracting: Train/Negative.zip      
 extracting: Train/Neutral.zip       
 extracting: Train/Positive.zip      
 extracting: Validation/Negative.zip  
 extracting: Validation/Neutral.zip  
 extracting: Validation/Positive.zip  
 extracting: ReadMe.txt              
 extracting: Test_Images.zip         


In [8]:
!unzip /content/drive/MyDrive/mtcnn.zip

Archive:  /content/drive/MyDrive/mtcnn.zip
 extracting: mtcnn/AlignedCroppedImages.zip  
 extracting: mtcnn/CroppedFaces.zip  
  inflating: mtcnn/FaceCoordinates.rar  
 extracting: mtcnn/FaceFeatures.zip  
  inflating: mtcnn/test_list         
  inflating: mtcnn/train_annotations.npz  
  inflating: mtcnn/val_annotations.npz  
  inflating: mtcnn/val_list          


In [9]:
!unzip /content/drive/MyDrive/FaceCoordinates.zip

Streaming output truncated to the last 5000 lines.
  inflating: FaceCoordinates/train/Positive/pos_413.npz  
  inflating: FaceCoordinates/train/Positive/pos_414.npz  
  inflating: FaceCoordinates/train/Positive/pos_415.npz  
  inflating: FaceCoordinates/train/Positive/pos_416.npz  
  inflating: FaceCoordinates/train/Positive/pos_417.npz  
  inflating: FaceCoordinates/train/Positive/pos_418.npz  
  inflating: FaceCoordinates/train/Positive/pos_419.npz  
  inflating: FaceCoordinates/train/Positive/pos_42.npz  
  inflating: FaceCoordinates/train/Positive/pos_420.npz  
  inflating: FaceCoordinates/train/Positive/pos_421.npz  
  inflating: FaceCoordinates/train/Positive/pos_422.npz  
  inflating: FaceCoordinates/train/Positive/pos_423.npz  
  inflating: FaceCoordinates/train/Positive/pos_424.npz  
  inflating: FaceCoordinates/train/Positive/pos_425.npz  
  inflating: FaceCoordinates/train/Positive/pos_426.npz  
  inflating: FaceCoordinates/train/Positive/pos_427.npz  
  inflating: FaceCoord

In [10]:
!unzip /content/Test_Images.zip

Archive:  /content/Test_Images.zip
   creating: test_shared/
  inflating: test_shared/test_1.jpg  
  inflating: test_shared/test_10.jpg  
  inflating: test_shared/test_100.jpg  
  inflating: test_shared/test_1000.jpg  
  inflating: test_shared/test_1001.jpg  
  inflating: test_shared/test_1002.jpg  
  inflating: test_shared/test_1003.jpg  
  inflating: test_shared/test_1004.jpg  
  inflating: test_shared/test_1005.jpg  
  inflating: test_shared/test_1006.jpg  
  inflating: test_shared/test_1007.jpg  
  inflating: test_shared/test_1008.jpg  
  inflating: test_shared/test_1009.jpg  
  inflating: test_shared/test_101.jpg  
  inflating: test_shared/test_1010.jpg  
  inflating: test_shared/test_1011.jpg  
  inflating: test_shared/test_1012.jpg  
  inflating: test_shared/test_1013.jpg  
  inflating: test_shared/test_1014.jpg  
  inflating: test_shared/test_1015.jpg  
  inflating: test_shared/test_1016.jpg  
  inflating: test_shared/test_1017.jpg  
  inflating: test_shared/test_1018.jpg  
  i

In [14]:
!unzip /content/mtcnn/CroppedFaces.zip

Streaming output truncated to the last 5000 lines.
  inflating: CroppedFaces/train/Positive/pos_413.npz  
  inflating: CroppedFaces/train/Positive/pos_414.npz  
  inflating: CroppedFaces/train/Positive/pos_415.npz  
  inflating: CroppedFaces/train/Positive/pos_416.npz  
  inflating: CroppedFaces/train/Positive/pos_417.npz  
  inflating: CroppedFaces/train/Positive/pos_418.npz  
  inflating: CroppedFaces/train/Positive/pos_419.npz  
  inflating: CroppedFaces/train/Positive/pos_42.npz  
  inflating: CroppedFaces/train/Positive/pos_420.npz  
  inflating: CroppedFaces/train/Positive/pos_421.npz  
  inflating: CroppedFaces/train/Positive/pos_422.npz  
  inflating: CroppedFaces/train/Positive/pos_423.npz  
  inflating: CroppedFaces/train/Positive/pos_424.npz  
  inflating: CroppedFaces/train/Positive/pos_425.npz  
  inflating: CroppedFaces/train/Positive/pos_426.npz  
  inflating: CroppedFaces/train/Positive/pos_427.npz  
  inflating: CroppedFaces/train/Positive/pos_428.npz  
  inflating: Cr

In [11]:
!mv "/content/test_shared" "/content/Dataset"

In [13]:
!mv "/content/Dataset/test_shared" "/content/Dataset/emotiw/test"

In [12]:
!mv "/content/FaceCoordinates" "/content/Dataset"

In [15]:
!mv "/content/CroppedFaces" "/content/Dataset"

In [22]:
dataset_path = '/content/Dataset/emotiw/test/'
face_coordinates_directory = '/content/Dataset/FaceCoordinates/'
processed_face_ex = '/content/Dataset/CroppedFaces/test/'

In [23]:

test_dataset = datasets.ImageFolder(dataset_path)

class_names = test_dataset.classes

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [24]:
class_names

['test_shared']

In [25]:
test_filelist = sorted(os.listdir(dataset_path + 'test_shared/'))

In [26]:

test_filelist = [x.split('.')[0] for x in test_filelist]

In [27]:
print(test_filelist[:10])

['test_1', 'test_10', 'test_100', 'test_1000', 'test_1001', 'test_1002', 'test_1003', 'test_1004', 'test_1005', 'test_1006']


In [28]:
print(len(test_dataset))

3011


In [29]:
for i in range(len(test_dataset)):
    print(test_filelist[i])
    image, label = test_dataset[i]

    face_list = []
    landmarks_new_coordinates = []

    if os.path.isfile(processed_face_ex + 'test/' + test_filelist[i] + '.npz'):
        print(test_filelist[i] + ' Already present')
        continue

    bbox_lm = np.load(face_coordinates_directory + 'test/' + test_filelist[i] +'.npz')
    bounding_boxes = bbox_lm['a']
    if bounding_boxes.size == 0 or (bounding_boxes[0] == 0).all():
        print("No bounding boxes for " + test_filelist[i] + ". Adding empty file for the same")
        np.savez(processed_face_ex + test_filelist[i], a = np.zeros(1), b = np.zeros(1))
        continue
    landmarks = bbox_lm['b']

    for j in range(len(bounding_boxes)):
        bbox_coordinates = bounding_boxes[j]
        landmark = landmarks[j]
        img_face = image.crop((bbox_coordinates[0], bbox_coordinates[1], bbox_coordinates[2], bbox_coordinates[3]))

        x = bbox_coordinates[0]
        y = bbox_coordinates[1]
        for k in range(5):
            landmark[k] -= x
            landmark[k+5] -= y
        img_face = np.array(img_face)
        landmark = np.array(landmark)


        if len(face_list) != 0:
            if img_face.shape[0] == face_list[-1].shape[0]:
                img_face = image.crop((bbox_coordinates[0] - 1, bbox_coordinates[1] - 1, bbox_coordinates[2], bbox_coordinates[3]))
                img_face = np.array(img_face)
                landmark +=1

        face_list.append(img_face)
        landmarks_new_coordinates.append(landmark)
    face_list = np.asarray(face_list)
    landmarks_new_coordinates = np.asarray(landmarks_new_coordinates)
    np.savez(processed_face_ex + test_filelist[i], a = face_list, b = landmarks_new_coordinates)

    print(i)

Streaming output truncated to the last 5000 lines.
test_1459
511
test_146
512
test_1460
513
test_1461
514
test_1462
515
test_1463
516
test_1464
517
test_1465
518
test_1466
519
test_1467
520
test_1468
521
test_1469
522
test_147
523
test_1470
524
test_1471
525
test_1472
526
test_1473
527
test_1474
528
test_1475
529
test_1476
530
test_1477
531
test_1478
532
test_1479
533
test_148
534
test_1480
535
test_1481
536
test_1482
537
test_1483
538
test_1484
539
test_1485
540
test_1486
541
test_1487
542
test_1488
543
test_1489
544
test_149
545
test_1490
546
test_1491
547
test_1492
548
test_1493
549
test_1494
550
test_1495
551
test_1496
552
test_1497
553
test_1498
554
test_1499
555
test_15
556
test_150
557
test_1500
558
test_1501
559
test_1502
560
test_1503
561
test_1504
562
test_1505
563
test_1506
564
test_1507
565
test_1508
566
test_1509
567
test_151
568
test_1510
569
test_1511
570
test_1512
571
test_1513
572
test_1514
573
test_1515
574
test_1516
575
test_1517
576
test_1518
577
test_1519
578
test_